# Chiffrement en LoRaWAN v1.1

<div class="alert alert-warning">
"Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing."
</div>

La norme : https://lora-alliance.org/sites/default/files/2018-04/lorawantm_specification_-v1.1.pdf

## Généralités : modes ABP ou OTAA

ABP ("Activation ny Personalization") : mode d'activation du device dans lequel la _DevAddr_ et les deux clés de sessions _NwkSKey_ et _AppSKey_ sont définies par le concepteur et stockées à la fois dans le device et dans le serveur de réseau. Il n'y a donc pas de négociation de clés de chiffrement comme dans le mode OTAA.

OTAA ("Over The Air Authentication") : le device contient en dur les _DevEUI_, _AppEUI_ et _AppKey_. Ces valeurs sont là aussi définies par le concepteur et stockées aussi dans le serveur de réseau. Lors de la procédure de _join_, le device va négocier avec le serveur de réseau la _DevAddr_ et les deux clés de sessions _NwkSKey_ et _AppSKey_.

Ensuite, que ce soit en ABP ou on OTAA, le chiffrement et la signature des messages se fera à partir des clés _NwkSKey_ et _AppSKey_

Ici, nous prenons un device dont voici les clés. Elles peuvent être lues dans l'interface du serveur de réseau. 

Le but ici est de faire tout le procédé de chiffrement utiiisé par le proctole LoRaWAN et, par la même occasion, de voir en détail comment est construit un paquet LoRaWAN.

In [1]:
DevAddr = "018e4b26"
NwkSkey = "026e26a58c4f234c3b9924f86dcad3a9"
AppSKey = "dd0a32bf8b4082bc4a0017e99c1517d6"

En nous abonnant au flux MQTT d'une passerelle, nous avons pu intercepter une des _phyPayLoad_ de notre device. Bien sûr, une partie de celle-ci a été chiffrée à partir des clés ci-dessus. C'est un message _uplink_. Nous allons donc essayer de déchiffrer "manuellement" le message...

In [2]:
#phyPayload = "QCZLjgGC4woFAwHcYJuX0c/5zJVZCw6brV8=" #OK
phyPayload = "QCZLjgGAngEBwwW4exCLA/TTNLkujCpy" #NOK

<div class="alert alert-success">
Attention, en LoRaWAN, les paquets sont codés en base 64 et nous devons travailler en binaire
</div>

D'où :

In [3]:
import base64
binData = base64.b64decode(phyPayload)
print(binData)

b'@&K\x8e\x01\x80\x9e\x01\x01\xc3\x05\xb8{\x10\x8b\x03\xf4\xd34\xb9.\x8c*r'


Inutile ici mais, affichons la représentation hexadécimale des données binaires. 

Chaque octet doit être converti dans sa représentation héxadécimale sur deux chiffres. 

La chaîne obtenue est donc deux fois plus longue que le binaire (cf https://docs.python.org/2/library/binascii.html)

In [4]:
print("phyPayload décodée : {}".format(binData.hex()))

phyPayload décodée : 40264b8e01809e0101c305b87b108b03f4d334b92e8c2a72


## **phyPayload** aux rayons X

Selon les spécifications LoRaWAN et dans notre cas particulier où nous n'analysons un paquet de "up link", le "train" de bits de la phyPayload, se décompose ainsi :

| MHDR          |     MACPayload     |        MIC |
| :------------ | :-------------:    | -------------: |
| 1 octets      |     1 octets mini  |        4 octets |


* **MHDR** (MAC Header). Spécifie le type de message (join request, data up, data down...) et la version du format de la trame LoRaWAN générée. Taille : 1 octet. Les 3 premiers codent le Mtype (le "Message type" : join request, accept, Unconfirmed Data Up,.. ). Pour de la donnée envoyée sans confirmation (Unconfirmed Data Up), ils valent 0b010 selon les spécifications.

* **MACPayload** : ces octets contiennent le frame header (FHDR), suivi de façon optionnelle du Fport et de notre message frame payload (FRMPayload) : 
 * **FHDR** : contient les adresses de source/destination et le compteur de messages (frame counter)
 * **Fport** : 0 si le message contient que des commandes MAC, autre valeur pour indiquer que les données dépendant d'une application
 * **FRMPayload** : contient notre message à déchiffrer !
 

* **MIC** : Message Integrity Code sur 4 octets. Il est calculé à partir de la concaténation des champs MHDR et MACPayload.

![Format message UpLinkk](LoRaWaN-uplink-message-format.png "Format trame uplink en LoRaWAN")

In [5]:
MHDR = binData[0]
Mtype = MHDR >> 5 # On récupère les 3 premiers bits de cet octet
MACPayload = binData[1:-4]
MIC = binData[-4:]

Quel est le type de message considéré ? Il faut savoir si le MType vaut 0b010 (soit 2 en décimal)

In [6]:
print(bin(Mtype))

0b10


## MACPayload

Comme vu ci-dessus, la **MACPayload** est structurée de la façon suivante :


| FHDR    |     FPort     |        FRMPayload |
| :------------ | :-------------: | -------------: |
| 7 à 22 octets       |     1 octets    | le reste... |

Selon les spécification, le **FHDR** peut contenir jusqu'à 22 octets car on peut y rajouter des options. 

Il va donc falloir connaître le nombre de ces options afin de savoir à partir de quel octets commencent **FRMPayload** !

In [7]:
print(MACPayload.hex())

264b8e01809e0101c305b87b108b03f4d334b9


### FHDR (Frame header) et FPort

Le champ **FHDR** contient les informations suivantes :

| DevAddr    |     FCtrl     |        FCnt |  FOpts
| :------------ | :-------------: | -------------: |-------------: |
| 4 octets       |     1 octet    | 2 octets | 0 à 15 octets |

On peut donc déjà récupérer les informations **DevAddr**, **FCtrl** et le compteur de trames (_frame_) **FCnt**.

* **DevAddr** est un entier 32 bits stocké sur 4 octets. Ligne 330 des specs : _The over-the-air octet order for all multi-octet fields is little endian_  Octet de poids faible au début.
* **FCtrl** est un octet dont les 3 derniers bits codent la longueur du champ **FOpts** (ligne 499 et 670).
* **FCnt** sur 2 octets est le compteur de messages. Ligne 653 : _Frame counters are 32bits wide, The FCntfield corresponds to the least-significant 16 bits_

Nous pourrons ainsi vérifier que l'on a bien intercepté un message venant de notre objet dont le _DevAddr_ est 018e4b26

In [8]:
DevAddr_FCtrl_FCnt = MACPayload[:8] # 7 premiers octets
print(DevAddr_FCtrl_FCnt)

b'&K\x8e\x01\x80\x9e\x01\x01'


In [9]:
import struct # convertion de données au format structure C en type python

DevAddr, FCtrl, FCnt, FOpts0 = struct.unpack("<IBHB", DevAddr_FCtrl_FCnt) # unpack nécessite 1 octet, on ajoute l'octet 0 de FOpts pour combler

# "<IBHB" signifie : 
# < : ordre des octets en little, I : unsigned int (4 octets), B : unsigned char (1 octet), H : unsigned short (2 octets)

print("  DevAddr: {:08x}    FCtrl: {:b}   FCnt: {:04x}   FOpts0: {:0x}".format(DevAddr, FCtrl, FCnt, FOpts0))

  DevAddr: 018e4b26    FCtrl: 10000000   FCnt: 019e   FOpts0: 1


On retrouve bien notre device. **FCnt** nous indique le numéro de trame interceptée.

Afin de déterminer à partir de quels octets de **MACPayload** commence **FRMPayload**, il nous faut la longueur du champ **FOpts**.

Comme vu, la longueur du champ **FOpts** est donnée par les 3 derniers bits de l'octet **FCtrl** :

In [10]:
print("FCtrl : {:b}".format(FCtrl))
print("FOpts a une taille de {} octets".format(FCtrl & 111))
cid , payload = struct.unpack("<BB", MACPayload[7:9])
print("cid: {:0x}       payload: {:0b}".format(cid, payload)) # cid=5 : RXParamSetupReq

FCtrl : 10000000
FOpts a une taille de 0 octets
cid: 1       payload: 11000011


On récupère la valeur de l'octet **FPort**. 

L'octet **FPort** doit être non-vide à partir du moment où il y a une **FRMPayload**.

En fonction de celle-ci on saura quelle clé de chiffrement utiliser pour déchiffrer notre message :

In [11]:
print(MACPayload[9])

5


Comme elle est différente de 0, la clé de chiffrement utilisée est la AppSKey (ligne 745).

### FRMPayload

Nous avons donc 4+1+2+2=9 octets de **FHDR** suivis de un octet pour **FPort**. 

On peut ainsi récupérer le champ **FRMPayload** qui est notre message chiffré.

In [12]:
lenFHDR_FPort = 4 + 1 + (FCtrl & 111) + 2
print("FHDR et FPort sont sur {} octets".format(lenFHDR_FPort))

FRMPayload = MACPayload[lenFHDR_FPort+1:]
print(str(FRMPayload.hex().upper())) # DC609B97D1CFF9CC95590B ou C305B87B108B03F4D334B9

FHDR et FPort sont sur 7 octets
C305B87B108B03F4D334B9


Selon les spécifications (ligne 741) l'algorithme de chiffrement utilisé est celui de la norme IEE802.15.4/2006, Annexe B qui utilise AES-128.

La clé utilisée pour chiffrer dépend da la valeur présente dans le champ FPort :
* FPort = 0 : la clé est NwkSKey
* FPort = [1..255] : la clé est AppSKey

Ici le FPort est différent de 0 donc la FRMPayload est chiffrée avec la **AppSKEY**.

## Chiffrement et déchiffrement de FRMPayload

### Généralités

Tout est expliqué en section 4.3.3 (ligne 738) des specs.

Dans toute la suite, on notera pld=FRMPayload. 

Nous avons vu que FPort est différent de zéro. La clé, notée $K$ dans la suite, est donc _AppSKey_. D'où :

In [13]:
pld = FRMPayload
K = AppSKey
blocksize = 16
print(len(pld),pld)

11 b'\xc3\x05\xb8{\x10\x8b\x03\xf4\xd34\xb9'


<div class="alert alert-success">
    Ici il est bien question de <b>déchiffrer</b> et <b>non décrypter</b> car nous avons la clé de chiffrement ! Voir <a>https://chiffrer.info/</a>
</div>



Selon les spécifications, chiffrement et déchiffrement du message s'opèrent en tronquant au _len(pld)_ premiers octets $(pld | pad_{16}) \oplus S$, où : 

* $pad_{16}$ ajoute des octets de zéro afin que la longueur totale de la donnée soit un multiple de 16. 
* $S$ est un concaténé de chiffrés AES de certaines données que nous allons calculer.

Selon les spécifications (p21) : $S=S_1 | S_2 | \dots | S_k$ où $k=\lceil len(pld)/16  \rceil$ (par exemple si len(pld) est de 18, k vaudra 2). 

Par exemple : $S_i=aes128encrypt(K,A_i)$. $A_i$ est une nouvelle suite de blocs définie par :

| 0x01      |   4 fois 0x00 |   Dir   | DevAddr  | FCntUP ou FCntDown | 0x00     |  i |
| :------:  | :-----------: | :----:   |:-------:  |:-------------:   |:--------: |:------------: |
| 1 octets  |     4 octets  | 1 octet | 4 octets | 4 octets           | 1 octets | 1 octet |

* **Dir** vaut 0 pour un uplink (1 pour un downlink)
* **FCntUP ou FCntDown**. Atttention, sur 4 octets alors que fCnt est sur 2 octets... Selon les spécifications (ligne 653) : the FCnt field corresponds to the least-significant 16 bits of the 32-bits frame counter (i.e., FCntUp for data frames sent uplink). Attention, nous sommes en little endian.

### Implémentation

Le chiffrement AES est fait en mode ECB (_The network server uses an AES decrypt operation in ECB mode_). 

Dans notre exemple, la payload a une taille de 12 octets. Inutile donc de découper en blocs de taille 16 et nous n'aurons besoin que de $s_1$.

#### Calcul de $(pld | pad_{16})$

Toujours selon les spécifications, on remplit avec des 0 de façons à avoir un bloc de taille 16. Notre bloc de départ ayant une taille de 12 :

In [14]:
#pld_padded = pld + bytes.fromhex("0000000000") # padding au marteau et burin !

pld_padded = pld + bytearray(blocksize-len(pld))


print(pld_padded, len(pld_padded), str(pld_padded.hex()).upper())

b'\xc3\x05\xb8{\x10\x8b\x03\xf4\xd34\xb9\x00\x00\x00\x00\x00' 16 C305B87B108B03F4D334B90000000000


#### Calcul de $A_1$ et de $S_1$

In [15]:
# FCnt as 32-bit, lsb first et little endian !
# on converti à nouveau en structure de type C (l.653)
FCntUP = struct.pack("<H",FCnt) + bytes.fromhex("0000") 
binDevAddr = struct.pack("<I", DevAddr) # /!\ endianness !

print(binDevAddr.hex()) # 018e4b26 en big
print(FCntUP.hex()) # 018e4b26 en big


A1 = bytes.fromhex("01") + bytes.fromhex("00")*4 + bytes.fromhex("00") + binDevAddr + FCntUP + bytes.fromhex("00") + bytes.fromhex("01") # tout en little endian visiblement !

print(A1, len(A1),A1.hex()) # 16 010000000000264b8e01e30a00000001

264b8e01
9e010000
b'\x01\x00\x00\x00\x00\x00&K\x8e\x01\x9e\x01\x00\x00\x00\x01' 16 010000000000264b8e019e0100000001


On voit que $A_i$ va dépendre du compteur de message. La "clé" utilsé pour chaque message sera donc différente.

On peut maintenant calculer $S_1$.

On rappelle que $S_i=aes128encrypt(K,A_i)$. D'où :


In [16]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

K = bytes.fromhex(AppSKey)

cipher = Cipher(algorithms.AES(K), modes.ECB() ,backend=default_backend())

encryptor = cipher.encryptor()
S1 = encryptor.update(A1)+encryptor.finalize()
print(len(S1),S1.hex(), type(S1))

16 9a4aed3d5fde4db09e7198cfa82f8303 <class 'bytes'>


On déchiffre alors la payload avec un $(pld | pad_{16})\oplus S_1$ :

In [17]:
#XOR ne marche qu'avec des entiers ou des str...
# int(hexchar, 16)
endianness = 'little' # semble inutile puisque ce sont des blocs de memoire

int_pld_padded = int.from_bytes(pld_padded, endianness) # bytes to int
print(int_pld_padded)
int_S1 = int.from_bytes(S1, endianness)
int_decrypted = int_pld_padded ^ int_S1
print(int_decrypted)

223900749531865444683089347
4668841523509102916641846347846471513


On convertit ensuite l'entier int_decrypted en octets que l'on tronque aux len(pld) premiers octets :

In [18]:
decrypted = int_decrypted.to_bytes(len(S1), endianness)
print(decrypted)
pld_clair = decrypted[:len(pld)]
print(pld_clair, endianness)

b'YOUFOUNDME!\xcf\xa8/\x83\x03'
b'YOUFOUNDME!' little


On peut tout convertir en base 64 pour retrouver la payload que l'on peut avoir en s'abonnant au flux MQTT côté application :

In [19]:
import binascii


pld_clair_base64 = base64.b64encode(pld_clair) # 6GJdtULRLotnNGg=
print(pld_clair_base64)
print(str(binascii.hexlify(pld_clair)).upper()[2:-1]) # en hexa
print("On doit avoir : \n594F55464F554E444D4521")

b'WU9VRk9VTkRNRSE='
594F55464F554E444D4521
On doit avoir : 
594F55464F554E444D4521


# Élements techniques

In [20]:
import base64, binascii, struct
octets = b'ABCDEFGHIJKL'
print(octets, len(octets))
print("\nbinaire vers base 64 :")
octets_base64 = base64.b64decode(octets)
print(octets_base64)
print("\nbinaire vers héxadécimal :" )
octets_hex = binascii.hexlify(octets)
print(octets_hex)
print("\nunpack")

a, b, c, d, e = struct.unpack("=IBHBI", octets) # https://docs.python.org/3/library/struct.htmlr
# "=IBHBI" = : ordre des bits natifs, I : unsigned int (4 octets), B : unsigned char (1 octet), H : unsigned short (2 octets), B : unsigned char (1 octet)... 
print("  a: {:x}    b: {:c}   c: {:020}   d: {:o} e: {:X}".format(a, b, c, d, e))

octet_new = struct.pack("<H",c)
print(octet_new)

u=binascii.hexlify(b"Et c'est le tout debut de ton atissage, petit Padawan! Courage...")

octets = binascii.unhexlify(u)

print(octets)

octets_base64 = binascii.b2a_base64(octets)

print(octets_base64)

b'ABCDEFGHIJKL' 12

binaire vers base 64 :
b'\x00\x10\x83\x10Q\x87 \x92\x8b'

binaire vers héxadécimal :
b'4142434445464748494a4b4c'

unpack
  a: 44434241    b: E   c: 00000000000000018246   d: 110 e: 4C4B4A49
b'FG'
b"Et c'est le tout debut de ton atissage, petit Padawan! Courage..."
b'RXQgYydlc3QgbGUgdG91dCBkZWJ1dCBkZSB0b24gYXRpc3NhZ2UsIHBldGl0IFBhZGF3YW4hIENvdXJhZ2UuLi4=\n'


# Ressources

https://lora-alliance.org/sites/default/files/2018-07/lorawan1.0.3.pdf

https://lorawan-packet-decoder-0ta6puiniaut.runkit.sh/